### Описание задачи

В некой стране Компания производит различные товары-сладости (SKU) и осуществляет продажу в 4х туристических городах (локациях), где количество и тип потребителей меняется от месяца к месяцу. Основную часть ассортимента Компании составляет шоколад, а основной объем продаж приходится на город Alpha City.

Компании доступно 2 источника информации, оба помесячные: внутренние данные по отгрузкам и розничным ценам в каждом из 4х городов, а также внешние данные, которые компания закупает у стороннего подрядчика, по количеству и типам клиентов в городах. 

Данные по продажам - sales.tsv  
Данные по потоку потребителей - customers.tsv

sku_id = уникальный идентификатор одной позиции товара (SKU = Stock Keeping Unit).  
ira = International Retail Association  = уникальный код локации согласно классификации международной ассоциации ретейлеров.  
product_category – категория продукта.  
item_per_bundle – количество единиц товара на 1 упаковку (например, бывают наборы по 3 шоколадки сразу).  
shape – форма упаковки.  
with_alcohol – содержит ли алкоголь в начинке (yes/no).  
filling – начинка.  
volume – объем продаж в штуках.  
price – цена 1 штуку товара в USD.  

Данные по типу и количеству клиентов (потоку клиентов), имеет глубокую экспертизу и дополнительные внешние источники (например, забронированные путевки), на основании которых предоставляет как фактические данные (до июня 2019 года включительно), так и достаточно точной прогноз на 3 месяца вперед (июль-сентября 2019)

Сейчас 01 июля 2019 года, Компания только что получила свежие данные по продажам до июня включительно. Вас, как специалиста по продвинутой аналитике, наняли для того, чтобы найти алгоритм, максимально точно описывающий поведение рынка, то есть зависимость продаж каждой позиции товара в каждой локации от различных факторов. 

### Задача 1

Спрогнозировать продажи по каждому SKU в ассортименте для каждой локации на 3 месяца вперед помесячно (июль, август, сентябрь) с предположением, что цены останутся неизменными по сравнению с последним месяцем.Точность оценивается по метрике SMAPE.  
Если для какого-то SKU в какой-то локации прогноз сделать невозможно, необходимо указать причину.

In [13]:
#pip install -r requirements.txt

In [8]:
import pandas as pd

In [14]:
#pip freeze > requirements.txt